# Loading dataset

In [75]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pylab

def mcol(v):
    return v.reshape((v.size, 1))

# Load of data function
def load(fname):
    DList = []
    labelsList = []

    with open(fname) as f:
        for line in f:
            try:
                attrs = line.split(',')[0:11]
                attrs = mcol(np.array([float(i) for i in attrs]))
                label = line.split(',')[-1].strip()
                DList.append(attrs)
                labelsList.append(label)
                
            except:
                pass

    return np.hstack(DList), np.array(labelsList, dtype=np.int32)



if __name__ == '__main__':
    
    ##Load train
    X_train, y_train = load('Train.txt')
    print(X_train.shape,y_train.shape)
#     plot_hist(X_train, y_train)
#     plot_scatter(X_train, y_train)
    #Load test
    X_test, y_test = load('Test.txt')
    print(X_test.shape,y_test.shape)
    
    X_train,y_train,X_test,y_test = Preprocess_data(X_train,y_train,X_test,y_test,outlierK=9)
    
    classifier = 'mvg'
    print(Compute_gaussian_model(X_train,y_train,X_test,y_test,classifier))

(11, 1839) (1839,)
(11, 1822) (1822,)
(8, 1813)
{'Sjoint': array([[4.71621403e+02, 8.30499915e-01, 5.63613936e-05, ...,
        1.42739038e+02, 1.96840943e+01, 2.41729527e+01],
       [1.74690046e+02, 4.62669093e-03, 2.05391687e-02, ...,
        4.76768986e-04, 9.52666421e-05, 1.87064739e+02]]), 'SMarginal': array([[6.46311449e+02, 8.35126606e-01, 2.05955301e-02, ...,
        1.42739515e+02, 1.96841895e+01, 2.11237692e+02]]), 'posterior': array([[7.29712283e-01, 9.94459893e-01, 2.73658378e-03, ...,
        9.99996660e-01, 9.99995160e-01, 1.14434846e-01],
       [2.70287717e-01, 5.54010721e-03, 9.97263416e-01, ...,
        3.34013316e-06, 4.83975436e-06, 8.85565154e-01]]), 'SPost': array([0, 0, 1, ..., 0, 0, 1], dtype=int64), 'acc': 0.851627137341423, 'err': 0.14837286265857696}


### Make outlier detection and feature selection
###### The classes are ordered and not balanced (e.g. there are many more normal wines than excellent or poor ones). Outlier detection algorithms could be used to detect the few excellent or poor wines. Also, we are not sure if all input variables are relevant. So it could be interesting to test feature selection methods.

## Preprocesing

In [1]:
def Preprocess_data(X_train,y_train,X_test,y_test,outlierK = 1.5):
    
    outliers = Outliers_IQR(X_train,outlierK)
    X_train_sin_Out = np.delete(X_train,outliers,1)
    y_train_sin_Out = np.delete(y_train,outliers,0)
    
    X_train_norm = standarization(X_train_sin_Out)
    
    for i in range (0,X_train_norm.shape[0]):
        DP, accumulated, P = PCA(X_train_norm,i)
        if(accumulated > 0.95):
            m = i
#             print(i)
            break
    
    outliers = Outliers_IQR(X_test,outlierK)
    X_test_sin_Out = np.delete(X_test,outliers,1)
    y_test_sin_Out = np.delete(y_test,outliers,0)
    
    X_test_norm = standarization(X_test_sin_Out)
    DP_test = np.dot(P.T,X_test_norm)
    print(DP_test.shape)
    
    return DP,y_train_sin_Out,DP_test,y_test_sin_Out
    

## Auxiliar pre-processing functions

In [74]:
# Outlier detection

def Outliers_IQR(x,k):
    """
    Compute the outliers based on IQR
    :param x: matrix of features
    :return: np array with the index of the outliers
    """
    q1 = np.percentile(x,25, axis = 1)
    q3 = np.percentile(x,75, axis = 1)
    IQR = q3 - q1
    IQR
    outMayor = q3 + (k * IQR)
    outMenor = q1 - (k *IQR)
    
    outliers = []
    count = 0
    for j in range (0,x.shape[1]):
        for i in range (0,x.shape[0]):
            if x[i,j]>outMayor[i] or x[i,j]<outMenor[i]:
#                 print(j) #Tengo que eliminar los ejemplos que aparecen en j (unique)
                outliers.append(j)
                count+=1
        
    return np.unique(outliers)



# Standardization of the data (fit them on the same range) since the ranges of the features are so different.

def standarization(D):
    """
    Compute the min-max standarization for the given data by rows
    :param D: matrix of data 
    :param m: number of PCA to compute
    :return: np array with the projection of the given data on the computed PCs
    """
    standarized = np.zeros((D.shape[0],D.shape[1]))

    for i in range (D.shape[0]):
        for j in range (D.shape[1]):
            standarized[i,j]= ((D[i][j]-D[i].min())/(D[i].max()-D[i].min()))
    return standarized


def PCA(D,m):
    """
    Compute the PCA for the given dara
    :param D: matrix of features
    :param m: number of PCA to compute
    :return: np array with the projection of the given data on the computed PCs
    """
    
# Extracting data mean
    mu = mcol(D.mean(1))
# Centering Data
    DC = D - mu
    C = np.dot(DC, DC.T)/ D.shape[1]
# Compute eigenvectors and eogenvalues

    U, s, Vt = np.linalg.svd(C)
#     print(s)

# We are going to use 2 PC and plot them
    P = U[:, 0:m]
    DP = np.dot(P.T, D)

    covarianza1 = covariance(DP)
    numerador = sum(covarianza1[i][i] for i in range(covarianza1.shape[1]))
    
    covarianzatotal = covariance(D)
    denominador = sum(covarianzatotal[i][i] for i in range(covarianzatotal.shape[1]))
    sumvarianze = (numerador/denominador)
    # Podemos ver aqui que hay un ejemplo que es un outlier, debemos eliminar primero los outliers antes de aplicar PCA
    return DP, sumvarianze, P


## Gaussian generative models

In [27]:
def Compute_gaussian_model(X_train,y_train,X_test,y_test,classifier):
    """
    Compute different gaussian models
    :param X_train: matrix of train features
    :param y_train: matrix of train labels
    :param X_test: matrix of test features
    :param y_test: matrix of test labels
    :classifier: ('mvg','naive','tied_naive'): gaussian model to be computed
    :return: np array with the index of the outliers
    """
    mean, covariance = compute_classifier(X_train, y_train, classifier=classifier)
    evaluation = evaluate(X_test, y_test, mean, covariance,
                              logarithmically=False)
    
    return(evaluation)


classifier = 'mvg'
Compute_gaussian_model(X_train,y_train,X_test,y_test,classifier)

{'Sjoint': array([[2.65993487e-02, 2.11713865e-04, 3.11686123e-07, ...,
         8.82435609e-02, 4.82349015e-02, 2.05346202e-05],
        [5.11706703e-02, 4.24100557e-07, 4.46636317e-07, ...,
         1.22529918e-06, 1.27758330e-07, 7.06205411e-04]]),
 'SMarginal': array([[7.77700190e-02, 2.12137965e-04, 7.58322440e-07, ...,
         8.82447862e-02, 4.82350293e-02, 7.26740031e-04]]),
 'posterior': array([[3.42025746e-01, 9.98000827e-01, 4.11020572e-01, ...,
         9.99986115e-01, 9.99997351e-01, 2.82557989e-02],
        [6.57974254e-01, 1.99917330e-03, 5.88979428e-01, ...,
         1.38852303e-05, 2.64866285e-06, 9.71744201e-01]]),
 'SPost': array([1, 0, 1, ..., 0, 0, 1], dtype=int64),
 'acc': 0.8150384193194292,
 'err': 0.18496158068057078}

## Also we have to do (maybe not do but explain) that is not so useful to use LDA on binary classification problems according what professor said in teory class. (Only 1 discriminant direction)

In [5]:
# Computing MVG and Log-Likelihood

MVG = logpdf_GAU_ND(DP,mean(DP),covariance(DP))
print(MVG)
LL = loglikelihood(DP,mean(DP),covariance(DP))
print(LL)

NameError: name 'logpdf_GAU_ND' is not defined

## Logistic regression (sin terminar)

In [6]:
from scipy.optimize import fmin_l_bfgs_b

def optimize_function_yz():
    x, f, d = fmin_l_bfgs_b(function_yz,
                            x0=np.array([0, 0]),
                            approx_grad=False,
                            iprint=1)
    print(f"x: {x}")
    print(f"f: {f}")
    print(f"d: {d}")


def function_yz(x: np.ndarray):
    y = x[0]
    z = x[1]
    f = np.power(y + 3, 2) + np.sin(y) + np.power(z + 1, 2)
    gradient = np.array([
        2 * (y + 3) + np.cos(y),
        2 * (z + 1)
    ])
    return f, gradient

def log_reg_func_factory(DTR: np.ndarray,
                         LTR: np.ndarray,
                         lambda_param: float):
    Z = 2 * LTR - 1
    n = DTR.shape[1]

    def log_reg_func(x):
        w = prml.vcol(x[0:-1])
        b = x[-1]
        regularization = np.power(np.linalg.norm(w), 2) * lambda_param / 2.0
        loss = np.sum(np.logaddexp(0, - Z * (w.T @ DTR + b))) / n
        return regularization + loss

    return log_reg_func

X_train = DP
y_train = y_train_sin_Out
X_test = DP_test
y_test = y_test_sin_Out

def train_binary_regression(X_train,y_train,X_test,y_test):
#     D, L = load_iris_binary()
#     (DTR, LTR), (DTE, LTE) = prml.split_data(D, L, 2.0 / 3.0)
    log_reg_func = log_reg_func_factory(X_train, y_train, lambda_param=0.01)
    omegas, f, d = fmin_l_bfgs_b(log_reg_func,
                                 x0=np.zeros(X_train.shape[0] + 1),
                                 approx_grad=True,
                                 iprint=1)
    print(f"omegas: {omegas}")
    print(f"f: {f}")
    print(f"d: {d}")
    w = omegas[0:-1]
    b = omegas[-1]
    prediction = prml.vcol(w).T @ X_test + b
    score = (np.ones((1, X_test.shape[1])) * prediction > 0).astype(int)
    print(f"scores: {1 - np.sum(score == prml.vrow(y_test)) / X_test.shape[1]}")
    
train_binary_regression(X_train,y_train,X_test,y_test)

NameError: name 'DP' is not defined

## Auxiliar functions to visualize data information graphs

In [ ]:
# Function for plotting histograms of each feature
def plot_hist(D, L):

    D0 = D[:, L==0]
    D1 = D[:, L==1]

    hFea = {
        0: 'Fixed acidity',
        1: 'Volatile acidity',
        2: 'Citric acid',
        3: 'Petal width',
        4: 'Residual sugar',
        5: 'Chlorides',
        6: 'Free sulfur dioxide',
        7: 'Total sulfur dioxide',
        8: 'Density',
        9: 'pH',
        10: 'Sulphates',
        11: 'Alcohol'
        }

    for dIdx in range(11):
        plt.figure()
        plt.xlabel(hFea[dIdx])
        plt.hist(D0[dIdx, :], bins = 10, density = True, alpha = 0.4, label = 'Good Wine')
        plt.hist(D1[dIdx, :], bins = 10, density = True, alpha = 0.4, label = 'Bad Wine')
        
        plt.legend()
        plt.tight_layout() # Use with non-default font size to keep axis label inside the figure
#         plt.savefig('hist_%d.pdf' % dIdx)
    plt.show()

# Function for plotting scatter of each pair os features
def plot_scatter(D, L):
    
    D0 = D[:, L==0]
    D1 = D[:, L==1]

    hFea = {
        0: 'Fixed acidity',
        1: 'Volatile acidity',
        2: 'Citric acid',
        3: 'Petal width',
        4: 'Residual sugar',
        5: 'Chlorides',
        6: 'Free sulfur dioxide',
        7: 'Total sulfur dioxide',
        8: 'Density',
        9: 'pH',
        10: 'Sulphates',
        11: 'Alcohol'
        }

    for dIdx1 in range(11):
        for dIdx2 in range(11):
            if dIdx1 == dIdx2:
                continue
            plt.figure()
            plt.xlabel(hFea[dIdx1])
            plt.ylabel(hFea[dIdx2])
            plt.scatter(D0[dIdx1, :], D0[dIdx2, :], label = 'Good Wine')
            plt.scatter(D1[dIdx1, :], D1[dIdx2, :], label = 'Bad Wine')
           
            plt.legend()
            plt.tight_layout() # Use with non-default font size to keep axis label inside the figure
#             plt.savefig('scatter_%d_%d.pdf' % (dIdx1, dIdx2))
        plt.show()

## PRML Imported directly

In [15]:
import math

import numpy as np
import scipy
import scipy.linalg

import prml


def vcol(vector: np.ndarray):
    """
    Converts vector into column vector
    :param vector: one dimensional vector np.ndarray (size,)
    :return: vector: two dimensional vector (size, 1)
    """
    return vector.reshape((vector.size, 1))


def vrow(vector: np.ndarray):
    """
    Converts vector into row vector
    :param vector: one dimensional vector np.ndarray (size,)
    :return: vector: two dimensional vector (1, size)
    """
    return vector.reshape((1, vector.size))


def load(file_name: str):
    """
    Return two np.ndarray objects with data and target values
    File must be in the format
    float, float, float, ..., str
    Having first values as integers/float for the features and last value as
    the target class
    :param file_name: name of a csv file
    :return: np.ndarray features x N , np.ndarray features,
    """
    array = []
    targets = []
    with open(file_name, 'r') as file:
        for raw_line in file:
            if raw_line:
                line = raw_line.split(',')
                raw_values, target = line[:-1], line[-1]
                values = [float(value) for value in raw_values]
                array.append(values)
                targets.append(target.strip())
    return np.array(array).T, np.array(targets)


def mean(matrix_d):
    """
    Compute mean of a matrix_d
    :param matrix_d: features x N data matrix
    :return: features x 1 column vector
    """
    return vcol(matrix_d.mean(1))


def covariance(matrix_d):
    """
    Compute covariance of a np.ndarray features x N
    :param matrix_d: features x N
    :return: covariance matrix : features x features
    """
    N = matrix_d.shape[1]
    mu = mean(matrix_d)
    data_centered = (matrix_d - mu)
    return data_centered @ data_centered.T / N


def covariance_between(matrix_d, matrix_l):
    """
    Return covariance between classes
    :param matrix_d: features x N
    :param matrix_l: classes vector
    :return: covariance matrix features x features
    """
    classes = set(matrix_l)
    features = matrix_d.shape[0]
    N = matrix_d.shape[1]
    s_b = np.zeros((features, features))
    mu = mean(matrix_d)
    for class_l in classes:
        d_class = matrix_d[:, matrix_l == class_l]
        nc = d_class.shape[1]
        mu_c = mean(d_class)
        classes_distance = mu_c - mu
        summation = np.multiply(nc, classes_distance) @ classes_distance.T
        s_b = s_b + summation
    return s_b / N


def covariance_within(matrix_d, matrix_l):
    classes = set(matrix_l)
    N = matrix_d.shape[1]
    features = matrix_d.shape[0]
    s_w = np.zeros((features, features))
    for class_l in classes:
        d_class = matrix_d[:, matrix_l == class_l]
        mu_c = mean(d_class)
        central_data = d_class - mu_c
        class_summation = central_data @ central_data.T
        s_w = s_w + class_summation
    return s_w / N


def compute_w(s_b, s_w, m):
    s, U = scipy.linalg.eigh(s_b, s_w)
    W = U[:, ::-1][:, 0:m]
    return W


def eigh(matrix_d):
    """
    Return eigen values and vectors using np. linalg.eigh but in desc order
    :param matrix_d: symmetric matrix
    :return: np.ndarray with eigen values, np.ndarray with eigen vectors
    """
    eig_values, eig_vectors = np.linalg.eigh(matrix_d)
    return eig_values[::-1], eig_vectors[:, ::-1]


def compute_pca(matrix_d, m):
    """
    Computes the Principal component Analysis of a Matrix
    :param matrix_d: (np.ndarray features, N) matrix
    :param m: number of components
    :return: Data projected over the m principal components
    """
    d_covariance = covariance(matrix_d)
    eig_values, eig_vectors = eigh(d_covariance)
    P = eig_vectors[:, 0:m]  # Eigen vectors to project
    DP = P.T @ matrix_d  # Matrix D projected on P
    return DP


def compute_w_2(s_b, s_w, m):
    U, s, _ = np.linalg.svd(s_w)
    P1 = np.dot(U * vrow(1.0 / (s ** 0.5)), U.T)
    Sbt = P1 @ s_b @ P1.T
    _, P2 = eigh(Sbt)
    P2 = P2[:, 0:m]
    W = P1.T @ P2
    return W


def compute_lda(matrix_d, targets, m):
    S_b = covariance_between(matrix_d, targets)
    S_w = covariance_within(matrix_d, targets)
    W = compute_w_2(S_b, S_w, m)
    DP = W.T @ matrix_d
    return DP


def logpdf_GAU_ND(X, mu, C):
    """
    Computes the Multivariate Gaussian Density
    :param X: matrix features x samples
    :param mu: mean
    :param C: empirical covariance
    :return:
    """
    M = X.shape[0]
    first_term = - 0.5 * M * np.log(2 * math.pi)
    centered_x = X - mu
    second_term = - 0.5 * np.linalg.slogdet(C)[1]
    third_term = - 0.5 * np.sum(
        (centered_x.T @ np.linalg.inv(C)) * centered_x.T,
        axis=1)
    return first_term + second_term + third_term


def loglikelihood(X, m, C):
    return logpdf_GAU_ND(X, m, C).sum(axis=0)

In [20]:
def training_50_50(classifier):
    D, L = load_iris()
    (DTR, LTR), (DTE, LTE) = split_db_2to1(D, L)
    mean, covariance = compute_classifier(DTR, LTR, classifier=classifier)
    return evaluate(DTE, LTE, mean, covariance, logarithmically=False)


def compute_classifier(DTR, LTR, classifier):
    data_by_classes = split_by_classes(DTR, LTR)
    mean_by_class, covariance_by_class = get_mean_and_covariance(
        data_by_classes)
    classifier_covariance = compute_classifier_covariance(covariance_by_class,
                                                          classifier, DTR, LTR)
    return mean_by_class, classifier_covariance


def compute_classifier_covariance(covariance, classifier, D, L):
    if classifier == 'mvg':
        return covariance
    elif classifier == 'naive':
        return diagonalize_covariance(covariance)
    elif classifier == 'tied':
        return [prml.covariance_within(D, L) for _ in range(3)]
    elif classifier == 'tied_naive':
        # TODO validate this
        covariance = prml.covariance(D)
        return diagonalize_covariance([covariance for _ in range(3)])


def evaluate(DTE, LTE, mean, covariance, logarithmically):
    S = get_score_matrix(DTE, mean, covariance,
                         logarithmically=logarithmically)
    prior = np.array([[np.count_nonzero(y_train == 0)/ y_train.shape[0]], [np.count_nonzero(y_train == 1)/ y_train.shape[0]]])
    SJoint = compute_join(S, prior, logarithmically=logarithmically)
    SMarginal = compute_marginal(SJoint, logarithmically=logarithmically)
    posterior = compute_posterior(SJoint, SMarginal,
                                  logarithmically=logarithmically)
    SPost = np.argmax(posterior, axis=0)
    accuracy = np.sum(SPost == LTE) / DTE.shape[1]
    err = 1.0 - accuracy
    return {
        'Sjoint': SJoint,
        'SMarginal': SMarginal,
        'posterior': posterior,
        'SPost': SPost,
        'acc': accuracy,
        'err': err
    }


def diagonalize_covariance(covariance_by_class):
    diagonalized_covariances = []
    for covariance in covariance_by_class:
        size = covariance.shape[0]
        diagonalized_covariances.append(covariance * np.identity(size))
    return diagonalized_covariances


def compute_posterior(SJoint, SMarginal, logarithmically=False):
    if logarithmically:
        return np.exp(SJoint - SMarginal)
    return SJoint / SMarginal


def compute_marginal(SJoint, logarithmically=False):
    if logarithmically:
        return prml.vrow(scipy.special.logsumexp(SJoint, axis=0))
    return prml.vrow(SJoint.sum(0))


def get_score_matrix(samples, mean_by_class, covariance_by_class,
                     logarithmically: False):
    samples_number = samples.shape[1]
    score = np.empty((0, samples_number))
    for mean, covariance in zip(mean_by_class, covariance_by_class):
        class_score = prml.logpdf_GAU_ND(samples, mean, covariance)
        if not logarithmically:
            class_score = np.exp(class_score)
        score = np.vstack([score, class_score])
    return score


def compute_join(score, class_probability, logarithmically=False):
    if logarithmically:
        return score + np.log(class_probability)
    return class_probability * score


def load_iris():
    iris = sklearn.datasets.load_iris()
    D, L = iris['data'].T, iris['target']
    return D, L


def get_mean_and_covariance(data_by_classes):
    mean = []
    covariance = []
    for class_data in data_by_classes:
        mean.append(prml.mean(class_data))
        covariance.append(prml.covariance(class_data))
    return mean, covariance


def split_by_classes(data, labels):
    classes = []
    for _class in set(labels):
        classes.append(data[:, labels == _class])
    return classes


def split_db_2to1(D: np.ndarray, L: np.ndarray, seed=0):
    nTrain = int(D.shape[1] * 2.0 / 3.0)
    np.random.seed(seed)
    idx = np.random.permutation(D.shape[1])
    idxTrain = idx[0:nTrain]
    idxTest = idx[nTrain:]

    DTR = D[:, idxTrain]
    DTE = D[:, idxTest]
    LTR = L[idxTrain]
    LTE = L[idxTest]
    return (DTR, LTR), (DTE, LTE)